In [1]:
########## auto login ############
import login
from login import *
import pandas as pd
from loguru import logger
logger.add("RSI_logs_{time}.log")
#kite.margins(segment = 'equity')['available']['live_balance']

NCtLNjdSsiiI15Zoch4WLxD0gCYdFE4M


1

In [2]:
logger.debug("Parameters : ")
#### NEW VERSION ###########
init_qty = 3
extended_quant = 1
stepmultiseries = [2,4,7,12,15,20,30]
basepricediffmulti = 0.0100
symbol_ip = 'USDINR20NOVFUT' # USDINR20OCTFUT / GBPINR20OCTFUT / EURINR20OCTFUT / JPYINR20OCTFUT
inst_token = 780803          # 690691(USD) / 490755(GBP) / 278019(EUR) / 690435(JPY)
order_type = 'NRML'
#############################
logger.debug("init_qty : "+str(init_qty))
logger.debug("extended_quant : "+str(extended_quant))
logger.debug("stepmultiseries : "+str(stepmultiseries))
logger.debug("basepricediffmulti : "+str(basepricediffmulti))
logger.debug("symbol_ip : "+str(symbol_ip))
logger.debug("inst_token : "+str(inst_token))
logger.debug("order_type : "+str(order_type))

2020-10-28 16:55:54.841 | DEBUG    | __main__:<module>:1 - Parameters : 
2020-10-28 16:55:54.844 | DEBUG    | __main__:<module>:11 - init_qty : 3
2020-10-28 16:55:54.846 | DEBUG    | __main__:<module>:12 - extended_quant : 1
2020-10-28 16:55:54.848 | DEBUG    | __main__:<module>:13 - stepmultiseries : [2, 4, 7, 12, 15, 20, 30]
2020-10-28 16:55:54.850 | DEBUG    | __main__:<module>:14 - basepricediffmulti : 0.01
2020-10-28 16:55:54.852 | DEBUG    | __main__:<module>:15 - symbol_ip : USDINR20NOVFUT
2020-10-28 16:55:54.853 | DEBUG    | __main__:<module>:16 - inst_token : 780803
2020-10-28 16:55:54.855 | DEBUG    | __main__:<module>:17 - order_type : NRML


In [3]:
######## CHECK QTY #############
myquantity = 0
def getquant():
    logger.debug("getquant : ")
    global myquantity, order_type, symbol_ip
    allpos = kite.positions()['net']
    for i in range(len(allpos)):
        if(allpos[i]['tradingsymbol']==symbol_ip and allpos[i]['product']== order_type):
            myquantity = allpos[i]['quantity']
            print('My Quantity : ' + str(allpos[i]['quantity']))
    logger.debug("My Quantity : "+str(myquantity))
    return myquantity

# getquant()

In [4]:
def get_completed_orders():
    global order_type
    logger.debug("get_completed_orders : ")
    myorders = kite.orders()
    completed_orders = []
    for i in range(len(myorders)):
        if(myorders[i]['status']=='COMPLETE' and
           myorders[i]['tradingsymbol']==symbol_ip and
           myorders[i]['product']==order_type):
            completed_orders.append(myorders[i])
    if(len(completed_orders)>1) :
        completed_orders[-1]['average_price']
        logger.debug("last_completed_orders : "+str(completed_orders[-1]))
        logger.debug("last_completed_orders_price : "+str(completed_orders[-1]['average_price']))
    else:
        logger.debug("completed_orders : "+str(completed_orders))
    
    return completed_orders
# get_completed_orders()

In [5]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
########## cancel orders ####################
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def cancel_all_pending_orders():
    global order_type
    logger.debug("cancel_all_pending_orders : ")
    myorder = kite.orders()
    pending_orders = []
    for i  in range(len(myorder)):
        if(myorder[i]['status']=='OPEN' and 
           myorder[i]['tradingsymbol']==symbol_ip and 
           myorder[i]['product']==order_type):
            pending_orders.append(myorder[i])
            kite.cancel_order(myorder[i]['variety'], myorder[i]['order_id'], parent_order_id=None)
            logger.debug("cancel_order @ "+'order_id : '+str(myorder[i]['order_id'])+' variety : '+str(myorder[i]['variety']))
# cancel_all_pending_orders()

In [6]:
############# get pending orders ################
def get_pending_orders():
    logger.debug("get_pending_orders : ")
    global pending_orders, order_type
    my_orders=kite.orders()
    pending_orders=[]
    for i in range(len(my_orders)):
        if(my_orders[i]['tradingsymbol']==symbol_ip and 
           my_orders[i]['status']=='OPEN' and 
           my_orders[i]['product']==order_type):
            pending_orders.append(my_orders[i])
    logger.debug("pending_orders : "+str(pending_orders))
    return pending_orders
# get_pending_orders()

In [7]:
############# calc rsi ################
import datetime
import ta
def calc_rsi():
    global RSI,inst_token
    old_lst=[]
    interval='minute'
    todaydt=datetime.date.today()
    hud_ago=todaydt-datetime.timedelta(days=6)
    to_date=datetime.date.isoformat(todaydt)
    from_date=datetime.date.isoformat(hud_ago)
    for i2 in range(1):
        new_lst = kite.historical_data(inst_token, from_date, to_date, interval,continuous=False)
        old_lst = new_lst + old_lst
        todaydt=todaydt-datetime.timedelta(days=7)
        hud_ago=hud_ago-datetime.timedelta(days=7)
        to_date=datetime.date.isoformat(todaydt)
        from_date=datetime.date.isoformat(hud_ago)
#         print(len(old_lst))
    mydf = pd.DataFrame(old_lst) 
    indicator_rsi = ta.momentum.rsi(close = mydf["close"], n=14, fillna=True)
    current_rsi = round(indicator_rsi[-1:].to_numpy()[0],4)
    RSI = current_rsi
    logger.debug("current_rsi : "+str(current_rsi))
    print('Current RSI : '+str(current_rsi))
    return current_rsi

    # indicator_sma = ta.trend.SMAIndicator(close = mydf["close"], n=14, fillna=True)
    # current_sma = round(indicator_sma.sma_indicator()[-1:].to_numpy()[0],4)
    # print('Current SMA : '+str(current_sma))

# calc_rsi()

In [8]:
################ update status #################
baseprice = 123.0
getquant()
def update_status():
    global init_qty,extended_quant,stepmultiseries,basepricediffmulti,baseprice,RSI,p1q,p1,p0q,p0
    myquantity = getquant()
    if(myquantity==0):
        if(RSI >60):
            logger.debug("RSI > 60 : "+str(RSI))
            is_done = ckqnt(0,-init_qty)
#             is_done = placeneworder(-init_qty,0)
            logger.debug("Order Placed @ "+str(-init_qty)+' market and Succeeded : '+str(is_done))
            if(is_done):
                completed_orders = get_completed_orders()
                baseprice = round(0.0025 * round(completed_orders[-1]['average_price'] /0.0025),4)
                calcsteps(-init_qty,baseprice)
                ckqnt(round(p0,4),p0q)
                logger.debug("Order Placed @ "+str(p0q)+' Limit : '+str(round(p0,4))+
                             ' and Succeeded : '+str(is_done))

        if(RSI <40):
            logger.debug("RSI < 40 : "+str(RSI))
            is_done = ckqnt(0,init_qty)
#             is_done = placeneworder(init_qty,0)
            if(is_done):
                completed_orders = get_completed_orders()
                baseprice =  round(0.0025 * round(completed_orders[-1]['average_price'] /0.0025),4)
                calcsteps(init_qty,baseprice)
                ckqnt(round(p0,4),p0q)
                logger.debug("Order Placed @ "+str(p0q)+' Limit : '+str(round(p0,4))+ 
                             ' and Succeeded : '+str(is_done))

    else:
        current_point = 0
        for i in range(1,len(stepmultiseries)+2):
            if(globals()['p'+str(i)+'q']==myquantity):
                current_point=i
                
        logger.debug('Current Point : '+str(current_point))
        next_point = 0
        contain_next_point=False
        if(current_point!=(len(stepmultiseries)+1)):
            next_point = current_point + 1
            logger.debug('Next Point : '+str(next_point))
            pending_orders = get_pending_orders()
            for i in range(len(pending_orders)):
                if(pending_orders[i]['tradingsymbol']==symbol_ip 
                   and pending_orders[i]['status']=='OPEN' 
                   and pending_orders[i]['price']== globals()['p'+str(next_point)]
                   and pending_orders[i]['quantity']== globals()['p'+str(next_point)+'q']):
                    contain_next_point = True

            if(!contain_next_point):
                ckqnt(globals()['p'+str(next_point)],globals()['p'+str(next_point)+'q'])
                logger.debug('ckqnt @ '+str(globals()['p'+str(next_point)])
                             +' : '+str(globals()['p'+str(next_point)+'q']))
                

        contain_next_point = False

# update_status()

2020-10-28 16:55:54.939 | DEBUG    | __main__:getquant:4 - getquant : 
2020-10-28 16:55:55.486 | DEBUG    | __main__:getquant:11 - My Quantity : 0


My Quantity : 0


In [9]:
############## CALC STEPS ###############
def calcsteps(currentqty, base_price):
    logger.debug('calc_steps : ('+str(currentqty)+','+str(base_price)+')'+' : ')
    '''calcsteps(currentqty, base_price)'''
    global p0,basepricediffmulti,p0q,extended_quant,p1,p1q
    if(currentqty <0):
        p0 = round(base_price - basepricediffmulti,4)
        p0q = 0
        print('p0q : '+str(p0q))
        print('p0 : '+str(p0))
        logger.debug('p0q : '+str(p0q))
        logger.debug('p0 : '+str(p0))
        print()
        print('base quant : '+str(currentqty))
        print('base_price : '+str(base_price))
        logger.debug('base quant : '+str(currentqty))
        logger.debug('base_price : '+str(base_price))
        print()
        p1q = currentqty
        p1 = round(base_price,4)
        print('p1q : '+str(p1q))
        logger.debug('p1q : '+str(p1q))
        print('p1 : '+str(p1))
        logger.debug('p1 : '+str(p1))
        for i  in range(1,len(stepmultiseries)+1):
            globals()['p'+str(i+1)+'q']=currentqty-((i)*extended_quant)
            print('p'+str(i+1)+'q : '+str(globals()['p'+str(i+1)+'q']))
            logger.debug('p'+str(i+1)+'q : '+str(globals()['p'+str(i+1)+'q']))
            globals()['p'+str(i+1)]= round(base_price+(basepricediffmulti*stepmultiseries[i-1]),4)
            print('p'+str(i+1)+' : '+str(globals()['p'+str(i+1)]))
            logger.debug('p'+str(i+1)+' : '+str(globals()['p'+str(i+1)]))
    if(currentqty >0):
        p0 = base_price + basepricediffmulti
        p0q = 0
        print('p0q : '+str(p0q))
        print('p0 : '+str(p0))
        logger.debug('p0q : '+str(p0q))
        logger.debug('p0 : '+str(p0))
        print()
        print('base quant : '+str(currentqty))
        print('base_price : '+str(base_price))
        logger.debug('base quant : '+str(currentqty))
        logger.debug('base_price : '+str(base_price))
        print()
        p1q = currentqty
        p1 = round(base_price,4)
        print('p1q : '+str(p1q))
        logger.debug('p1q : '+str(p1q))
        print('p1 : '+str(p1))
        logger.debug('p1 : '+str(p1))
        
        for i  in range(1,len(stepmultiseries)+1):
            globals()['p'+str(i+1)+'q']=currentqty+((i)*extended_quant)
            print('p'+str(i+1)+'q : '+str(globals()['p'+str(i+1)+'q']))
            logger.debug('p'+str(i+1)+'q : '+str(globals()['p'+str(i+1)+'q']))
            globals()['p'+str(i+1)]= round(base_price-(basepricediffmulti*stepmultiseries[i-1]),4)
            print('p'+str(i+1)+' : '+str(globals()['p'+str(i+1)]))
            logger.debug('p'+str(i+1)+' : '+str(globals()['p'+str(i+1)]))

print('Quantity Series : '+str(stepmultiseries))
print()
# calcsteps(init_qty,74.00)

Quantity Series : [2, 4, 7, 12, 15, 20, 30]



In [10]:
########### place order ###########
stopbuy = False
stopsell = False

def placeneworder(quantdiff,price_ip):
    '''placeneworder(quantdiff,price_ip)'''
    logger.debug('placeneworder @ ('+str(quantdiff)+','+str(price_ip)+')')
    global stopbuy,stopsell,order_type,symbol_ip
    if(quantdiff>0 and stopbuy == False):
        try:
            order_id= kite.place_order(tradingsymbol=symbol_ip,
                        exchange=kite.EXCHANGE_CDS,
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        quantity=abs(quantdiff),
                        price=price_ip,
                        order_type=kite.ORDER_TYPE_MARKET if price_ip==0 else kite.ORDER_TYPE_LIMIT,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            stopsell = False
            logger.debug('Order Successfully Placed @ '+str(order_type)+' '+str(quantdiff)+' '+str(price_ip))
            getquant()
            return True
        except Exception as e:
            stopbuy = False
            print(e)
            logger.debug('Order Rejected For @ '+str(order_type)+' '+str(quantdiff)+' '+str(price_ip))
            getquant()
            return False
    if(quantdiff<0 and stopsell == False):
        try:
            order_id= kite.place_order(tradingsymbol=symbol_ip,
                        exchange=kite.EXCHANGE_CDS,
                        transaction_type=kite.TRANSACTION_TYPE_SELL,
                        quantity=abs(quantdiff),
                        price=price_ip,
                        order_type=kite.ORDER_TYPE_MARKET if price_ip==0 else kite.ORDER_TYPE_LIMIT,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            stopbuy = False
            logger.debug('Order Successfully Placed @ '+str(order_type)+' '+str(quantdiff)+' '+str(price_ip))
            getquant()
            return True
        except Exception as e:
            stopsell = True
            print(e)
            logger.debug('Order Rejected For @ '+str(order_type)+' '+str(quantdiff)+' '+str(price_ip))
            getquant()
            return False

def ckqnt (orderprice,orderquant):
    logger.debug('ckqnt @ ('+str(orderprice)+','+str(orderquant)+') in ckqnt')
    global myquantity
    quantdiff = orderquant - myquantity
    logger.debug(str(quantdiff)+' = '+str(orderquant)+'-'+str(myquantity)+ ' in ckqnt')
    is_done = placeneworder(quantdiff,orderprice)
    return is_done

def calc_and_update():
    logger.debug('calc_and_update : '+ ' in calc_and_update')
    try:
        calc_rsi()
    except:
        print('calc_rsi failed'+ ' in calc_and_update')
    update_status()

In [11]:
############# start trading ############
import threading
import logging
from kiteconnect import KiteTicker
logging.basicConfig(level=logging.DEBUG)
# kws = KiteTicker("w19o0chuo929jxkp", "eA5B5OJQNOtZ0OYihkBmYw7Ke3B9pmCC")

def on_ticks(ws, ticks):
    global pivot_price, moving_pivot_on
    this_sec = datetime.datetime.now().second
    last_sec = 0
    if(this_sec%10==0 and last_sec!=this_sec):
            last_sec = this_sec
            print('in the thread : ')
            x = threading.Thread(target=calc_and_update, args=())
            x.start()
#             x.join()

    print('=====================================')
    print( 'LTP : ' + str(ticks[0]['last_price']))
#     print(datetime.datetime.now().second)
#     print('=====================================')
#     print()

def on_connect(ws, response):
    global inst_token
    ws.subscribe([inst_token])
    ws.set_mode(ws.MODE_FULL, [inst_token])

def on_close(ws, code, reason):
    ws.stop()
    
def on_error(ws, code, reason):
    logging.error("closed connection on error: {} {}".format(code, reason))

def on_noreconnect(ws):
    logging.error("Reconnecting the websocket failed")

def on_reconnect(ws, attempt_count):
    logging.debug("Reconnecting the websocket: {}".format(attempt_count))

def on_order_update(ws, data):
    logger.debug('on_order_update : ')
    global baseprice,symbol_ip,myquantity,init_qty,current_point
    print("order update: ", data)
#     getquant()
#     if(data['tradingsymbol'] == symbol_ip and (myquantity!=init_qty and myquantity!=-init_qty)):
#         current_point = 0
#         for i in range(1,len(stepmultiseries)+1):
#             if(globals()['p'+str(i)+'q']==myquantity):
#                 current_point=i
#         logging.debug('current_point : '+str(current_point))
#         update_status()
          
kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

kws.connect()

LTP : 74.0625
LTP : 74.0625
LTP : 74.0625
LTP : 74.0625
LTP : 74.0625
LTP : 74.0625
LTP : 74.0625
LTP : 74.0625
LTP : 74.0625
LTP : 74.065
LTP : 74.065
LTP : 74.065
LTP : 74.0625
LTP : 74.0625
LTP : 74.065
LTP : 74.065
LTP : 74.0625


2020-10-28 16:56:10.505 | DEBUG    | __main__:calc_and_update:59 - calc_and_update :  in calc_and_update


in the thread : 
LTP : 74.0625


2020-10-28 16:56:11.235 | DEBUG    | __main__:calc_rsi:24 - current_rsi : 37.1133
2020-10-28 16:56:11.237 | DEBUG    | __main__:getquant:4 - getquant : 


Current RSI : 37.1133


...

2020-10-28 16:56:36.007 | DEBUG    | __main__:on_order_update:43 - on_order_update : 
2020-10-28 16:56:36.106 | DEBUG    | __main__:placeneworder:20 - Order Successfully Placed @ NRML 1 74.045
2020-10-28 16:56:36.107 | DEBUG    | __main__:getquant:4 - getquant : 


order update:  {'account_id': 'FC5917', 'unfilled_quantity': 0, 'checksum': '', 'placed_by': 'FC5917', 'order_id': '201028004312502', 'exchange_order_id': '1000000002273420', 'parent_order_id': None, 'status': 'OPEN', 'status_message': None, 'status_message_raw': None, 'order_timestamp': '2020-10-28 16:56:36', 'exchange_update_timestamp': '2020-10-28 16:56:36', 'exchange_timestamp': '2020-10-28 16:56:36', 'variety': 'regular', 'exchange': 'CDS', 'tradingsymbol': 'USDINR20NOVFUT', 'instrument_token': 780803, 'order_type': 'LIMIT', 'transaction_type': 'BUY', 'validity': 'DAY', 'product': 'NRML', 'quantity': 1, 'disclosed_quantity': 0, 'price': 74.045, 'trigger_price': 0, 'average_price': 0, 'filled_quantity': 0, 'pending_quantity': 1, 'cancelled_quantity': 0, 'market_protection': 0, 'meta': {}, 'tag': None, 'guid': '18679XvZaqspjNw6kK'}


2020-10-28 16:56:36.436 | DEBUG    | __main__:getquant:11 - My Quantity : 3
2020-10-28 16:56:36.438 | DEBUG    | __main__:update_status:54 - ckqnt @ 74.045 : 4


LTP : 74.065
My Quantity : 3
LTP : 74.0625


ERROR:kiteconnect.ticker:Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
ERROR:root:closed connection on error: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
ERROR:kiteconnect.ticker:Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Unhandled Error
Traceback (most recent call last):
  File "/home/hemang/.local/lib/python3.8/site-packages/twisted/internet/defer.py", line 311, in addCallbacks
    self._runCallbacks()
  File "/home/hemang/.local/lib/python3.8/site-packages/twisted/internet/defer.py", line 654, in _runCallbacks
    current.result = callback(current.result, *args, **kw)
  File "/home/hemang/.local/lib/python3.8/site-packages/twisted/internet/base.py", line 447, in _continueFiring
    callable(*args, **kwargs)
  File "/home/hemang/.lo